# Q2

#### Let’s consider a simple demonstration of MCMC sampling in a setting where conjugacy is actually possible – normal likelihoods with a known population variance, for which the prior is another normal distribution

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib

# Part(a)

#### Write a function to calculate the Bayesian posterior probability given 50 new data samples drawn from a normal distribution with mean 10 and SD 5, assuming a normal prior with mean 25 and s.d. 5. Plot the pdfs of the prior, the likelihood and the posterior distributions. Explain how you derive the likelihood from the data.

In [ ]:
'''
this takes likelihood and prior parameters as input
and returns posterior mean and sd using the formulas 
their derivation is explained in markdown comments
'''
def posterior(n_samples=50,lik_mean=10,lik_sd=5,prior_mean=25,prior_sd=5):
    # to reproduce random state is fixed
    np.random.seed(1)
    samples=np.random.normal(lik_mean,lik_sd,n_samples)
   
    # formulas derivations for these calc explained in markdowns

    post_sd=np.sqrt((lik_sd**2*prior_sd**2)/(lik_sd**2+prior_sd**2))

    post_mean=((lik_sd**2)*(prior_mean)+(n_samples*prior_sd**2)*(np.mean(samples)))/((n_samples*prior_sd**2)+lik_sd**2)

    return {'post_mean':round(post_mean,2),'post_sd':round(post_sd,3)}

In [ ]:
'''
takes mean and sd of an normal dist
and returns 
    - a equi space points around mean with 3*sd width and 
    - its pdf value for given normal dist in a dictionary 
'''
def norm_data(mean,sd):
    x=np.linspace(-3*sd+mean,mean+3*sd,100)
    norm_pdf = (1/(np.sqrt(2*np.pi) * sd )) * np.exp(-0.5*((x-mean)/sd)**2)
    return {'x':list(x),'pdf':list(norm_pdf)}

#### To plot lik-prior-posterior data 

In [ ]:
# init given info
n_samples=200
lik_mean=10
lik_sd=5
prior_mean=25
prior_sd=5

# call posterior function to get params values with default values
post=posterior()

# get x-y data for plots
lik_data=norm_data(lik_mean,lik_sd)
prior_data=norm_data(prior_mean,prior_sd)
post_data=norm_data(post['post_mean'],post['post_sd'])

# plot pdf compare graph
plt.plot(lik_data['x'],lik_data['pdf'],label='lik',color='black')
plt.plot(prior_data['x'],prior_data['pdf'],label='prior',color='red')
plt.plot(post_data['x'],post_data['pdf'],label='posterior',color='green')
plt.legend()
plt.title("Density plot for lik, prior, and posterior")
plt.ylabel('pdf-values')
plt.xlabel('x')
plt.show()

### observations:
- for these specific values
    - posterior mean has shifted in the direction of prior only by a very small amount(0.17) and 
    - sd has dropped to 3.536 from 5 that was for likelihood and prior.

### Explain Likelihood Derivation from Data

We basically have to derive the likehood from the samples we generated in `posterior` function. We will consider a general situation with mean $\mu$ and sd $\sigma$. Our data is just a specific case of that.

Now, consider N i.i.d. scalar obs $X = \{x_1 , x_2 , \ldots , x_N \}$ drawn from a normal distribution with mean $\mu$ and sd $\sigma$. So, $\forall x \in \{1,\ldots,N\}$ we have :
$$
p(x_n|\mu,\sigma^2)=\mathcal{N}(x_n|\mu,\sigma^2)
$$

Or we can write,
$$
p(x_n|\mu,\sigma^2)  = \frac{1}{\sqrt{2\pi \sigma^2}}exp\left[-\frac{(x_n-\mu)^2}{2\sigma^2}\right]
$$

Using above we can give joint likelihood simply as their product(as they are iid),
$$
p(X|\mu,\sigma^2)  = \prod_{n=1}^N p(x_n|\mu,\sigma^2)
$$

Full expanded version can be given as,
$$
p(X|\mu,\sigma^2)  = \frac{1}{(2\pi)^{n/2} \sigma^n}exp\left[-\frac{1}{2\sigma^2} \sum_{n=1}^N (x_n-\mu)^2\right]
$$


### posterior derivation formulas explained

To get posterior for `mean`($\mu$) from lik and prior we can use bayes formula,
$$
p(\mu|X)=\frac{p(X|\mu)p(\mu)}{p(X)}=\frac{p(X|\mu)p(\mu)}{\int p(X|\mu)p(\mu)}
$$

Up to proportinality we write(as integral is just a constant ),
$$
p(\mu|X) \propto p(X|\mu)p(\mu)
$$

**`Lik:`**

Given in above section. We will ignore constants(w.r.t. to $\mu$) though.

**`Prior:`**

For general case when prior for $\mu$ is a Guassian with mean $\mu_0$ and sd $\sigma_0$, we can write prior distribution,
$$
p(\mu|\mu_0,\sigma_0^2)  = \frac{1}{\sqrt{2\pi \sigma_0^2}}exp\left[-\frac{(\mu-\mu_0)^2}{2\sigma_0^2}\right]
$$

**`Posterior:`**

Use Baues rule upto constant,
$$
p(\mu|X) \propto \left(
    \frac{1}{(2\pi)^{n/2} \sigma^n}exp\left[-\frac{1}{2\sigma^2} \sum_{n=1}^N (x_n-\mu)^2\right]\right) \left(\frac{1}{\sqrt{2\pi \sigma_0^2}}exp\left[-\frac{(\mu-\mu_0)^2}{2\sigma_0^2}\right]\right)
$$

WE also can ignore other constants like $\sigma$(as it si known) and $\sigma_0$,
$$
p(\mu|X) \propto \left(
    exp\left[-\frac{1}{2\sigma^2} \sum_{n=1}^N (x_n-\mu)^2\right]\right) \left(exp\left[-\frac{(\mu-\mu_0)^2}{2\sigma_0^2}\right]\right)
$$

After using square trick and comaring with
$$
p(\mu|X)  \propto exp\left[-\frac{(\mu-\mu_N)^2}{2\sigma_N^2}\right]
$$

we get Gaussian posterior’s precision as the sum of the prior’s precision and sum of the noise
precisions of all the observations,
$$
\frac{1}{\sigma_N^2}=\frac{1}{\sigma_0^z} + \frac{N}{\sigma^2}
$$

and Gaussian posterior’s mean is a convex combination of prior’s mean and the MLE solution,
$$
\mu_N=\frac{\mu_0 \sigma^2+\bar{x}N\sigma_0^2}{N\sigma_0^2+\sigma^2}
$$

Here: $\bar{x}=\frac{\sum_{n=1}^N x_n}{N}$

NOTE: these formulas have been used in `posterior()` function to get posterior mean and SD parameter values


# Part(b)

#### Implement the Metropolis algorithm from the lecture slides to estimate the posterior distribution given the same prior and data and show that it converges to the analytic posterior by plotting a histogram of samples from the distribution alongside the analytic posterior distribution.  Assume whatever SD (width) you want for the proposal distribution.

### Algorithm Derivation

Now suppose that we did not know about square trick then it won't have been easy to do this integral and compute above posterior in closed and nice form. And that happens all the time in real worl dproblem. So, we are going to experiment with this problem where we suppose we don't know how to solve above problem and we will use Monte Carlo Markov Chains to get enough samples from the posterior distribution formula up to prop constant and we call this our `Target` distribution from which we need samples but we can not as it is not a standard known dist. That is why we use a proposal dist(TBD) that conditions on the immediately previous $\mu$ value (say a Gaussian).

Here we have to decide what to use as our proposal(jump) distribution. Since we know our posterior(target dist) has support all over the real line, we can consider Univariate Norrmal Distribution for this case as it does cover real line and also simple sample from. 

So our proposal distribution is a Normal Distribution. 

**`Algo Steps:`** A general MH algo steps are,

Let $x_0 =init$ and for a $k^{th}$ step when $x_k=x$ and to obtain $x_{k+1}$ we do these steps:
1. Sample a new possible proposal $y$ from the jump distribution that conditions on the immediately previous (that is $x$ here) value.
2. Calculate the ratio of the proposed posterior distribution to the current one. The ratios is given by
$$
r=\frac{f(y)g(x|y)}{f(x)g(y|x)}
$$

But since we are using guassian dist as our proposal we have $g(x|y)=g(y|x)$. meaning it is symmatric. We have same probability of going from x to y as y to x. So, remember here f is our posterior distribution upto prop constant,
$$
r=\frac{f(y)}{f(x)}
$$

3. Sample a random number a between 0 and 1 using `Uniform dist` --> $U(0,1)$. Say the sample is `a`
4. If r > a, accept the proposed $y$, otherwise stick with the earlier $y$
5. This gives one sample from the target distribution. We can repeat above steps to get more samples.

In [ ]:
'''
function to compute target dist's probability value in each run
takes a point from support space(real line here)
returns log of the posterior dist at that point
we have used log for computational stability(it does not affect inequality condition for choosing samples from MCMC as it is monotionic)
'''
def log_post(x,mean,sd):
    # log og posterior up to prop constant
    log_p=-((x-mean)/sd)**2
    
    return log_p

In [ ]:
'''
takes hyperparameter values and
returns mcmc samples
other things explained in func comments
'''
no_of_samples=10000
def mh(jump_sd=5,init=10,no_of_samples=no_of_samples):
    global post
    global 
    # to reproduce results
    np.random.seed(1)

    # list to store samples
    mcmc_samples=list()

    x_curr=init
    '''
    x_curr=10 # inital value --> hyper-para
    no_of_samples=10000 # hyper-para
    jump_sd=5 # hyper-para
    '''

    acc_count=0
    for i in range(no_of_samples):
        # propose a new value from jump dist
        y_prop=np.random.normal(x_curr,jump_sd)

        # calc ratio( in log ) --> posterior parameters
        r=(log_post(y_prop,post['post_mean'],post['post_sd']))-(log_post(x_curr,post['post_mean'],post['post_sd']))

        # propose random sample from uniform
        u=np.random.uniform(0,1)

        # accept-reject condition for MH
        if r > np.log(u):
            mcmc_samples.append(y_prop)
            # update value of current sample
            x_curr=y_prop
            # update acceptance count
            acc_count+=1
        else:
            mcmc_samples.append(x_curr)
    
    print(f'acceptance rate is {acc_count*100/no_of_samples} with jump SD={jump_sd}')
    return mcmc_samples


# actual posterior params that we got in first que is used
actual_samples=np.random.normal(post['post_mean'],post['post_sd'],no_of_samples)
# func return
mcmc_samples=mh()
# plot hists to compare
plt.hist(mcmc_samples,label='estimate(MCMC)',bins=100,alpha=0.3,density=True)
plt.hist(actual_samples,label='analytic',bins=100,alpha=0.3,density=True)
plt.legend()
plt.title("hist plot for mcmc and posterior(actual) samples")
plt.xlabel('x')
plt.show()

# Part(c)

#### How does the speed of convergence of the sampling depend on the proposal width? Is there an optimal proposal width that would work best? Demonstrate the consequences of using sub-optimal proposal width and terminating sampling too soon.

- todos


In [167]:
plt.acorr(mcmc_samples,maxlags=1000)

(array([-1000,  -999,  -998, ...,   998,   999,  1000]),
 array([0.84942738, 0.84965309, 0.85049724, ..., 0.85049724, 0.84965309,
        0.84942738]),
 <matplotlib.lines.Line2D at 0x7efe2674df70>)

### Observations

From the above example it is clear that the quality of the MCMC sampler is determined by the choice of the variance in the proposal distribution(SD).
- If SD is too large, then we will propose values potentially too far away, which in principle would be good, but this means a lot of the values will be rejected. That
means we will stay where we are quite often, increasing the correlation in the sample.
- If SD is too small, then we will make tiny moves implying we may accept a lot of them, but the samples obtained will be quite dependent.
